In [1]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from pathlib import Path
from research_tools.gpu import get_gpus_available

load_dotenv()


hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

n_gpus = 1

gpus_available = get_gpus_available()
n_gpus = min(n_gpus, len(gpus_available))
gpus = gpus_available[:n_gpus]

assert n_gpus > 0, "No GPUs available"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpus])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq

dataset_config = DATASETS_DICT[Datasets.WMDP]

# retain_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

data_dir = Path("../data")


def get_dataset(train_files: List[str], val_files: List[str], max_length: int):
    train = local_load_dataset(data_dir, train_files)
    val = local_load_dataset(data_dir, val_files)
    train_records = process_corpus(train, tokenizer, max_length)
    val_records = process_mcq(val, tokenizer, max_length)
    mcq_records = process_mcq(val, tokenizer, max_length, expand_choices=False)
    return {
        "corpus": train_records,
        "mcq": mcq_records,
        "val": val_records,
    }


n_val_files = 4
max_length = 512

unlearn_files = dataset_config["unlearn_files"]
val_unlearn_files = dataset_config["val_unlearn_files"]
retain_files = dataset_config["retain_files"]
val_retain_files = dataset_config["val_retain_files"]

store = {
    "A": get_dataset(
        unlearn_files[:n_val_files], val_unlearn_files[:n_val_files], max_length
    ),
    "B": get_dataset(
        unlearn_files[n_val_files:], val_unlearn_files[n_val_files:], max_length
    ),
    "retain": get_dataset(retain_files, val_retain_files, max_length),
}

Filter:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Filter:   0%|          | 0/471 [00:00<?, ? examples/s]

Map:   0%|          | 0/471 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [4]:
VALID_DATASETS = [d.name for d in Datasets]
print(VALID_DATASETS)

['MMLU', 'YEARS', 'WMDP', 'RANDOM_BD']


In [7]:
eval_dict = {k: v["val"] for k, v in store.items()}

In [5]:
import wandb

wandb.login()

config = {
    "model_id": model_id,
    "magnitude": 6.5,
    "lr": 1e-5,
    "n_epochs": 12,
    "forget_alphas": {"A": 0.39422},
    "retain_alphas": {"B": 13.51609, "retain": 1},
    "datasets_config": dataset_config,
}


run = wandb.init(
    project="relearn", config=config, tags=["rmu", "debug"], entity="12tqian"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian. Use `wandb login --relogin` to force relogin


In [6]:
from relearn.unlearn.rmu import train_rmu

config = run.config

model = train_rmu(
    model,
    {"A": store["A"]["corpus"]},
    {"B": store["B"]["corpus"], "retain": store["retain"]["corpus"]},
    eval_records_dict=eval_dict,
    n_epochs=config["n_epochs"],
    magnitude=config["magnitude"],
    lr=config["lr"],
    forget_alphas=config["forget_alphas"],
    retain_alphas=config["retain_alphas"],
    eval_at_start=False,
    max_batches=None,
    verbose=True,
    debug=False,
    tokenizer=tokenizer,
)

In [7]:
from pathlib import Path

path = Path("../models/wmdp") / "unlearn_A_retain_B"
# os.makedirs(path, exist_ok=True)
# model.save_pretrained(path)

# torch.save(model.state_dict(), path)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16).to(
    device
)

# assert False

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
from relearn.unlearn.rmu import train_rmu


model = train_rmu(
    model,
    {"B": store["B"]["corpus"]},
    {"retain": store["retain"]["corpus"]},
    eval_records_dict=eval_dict,
    n_epochs=5,
    magnitude=6.5,
    lr=1e-5,
    forget_alphas={
        "B": 1,
    },
    retain_alphas={
        "retain": 1,
    },
    eval_at_start=False,
    verbose=True,
    max_batches=10,
    tokenizer=tokenizer,
)

  0%|          | 0/10 [00:00<?, ?it/s]/mnt/align1_drive/tcqian/unlearning_order/src/relearn/unlearn/rmu/unlearn.py:253: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  forget_loss = torch.nn.functional.mse_loss(
100%|██████████| 393/393 [00:42<00:00,  9.25it/s]


In [9]:
from relearn.attacks.rtt import train_rtt

# relearm only A
model = train_rtt(
    model,
    10,
    store["A"]["mcq"],
    eval_dict,
    batch_size=2,
    lr=1e-6,
    eval_at_start=True,
    grad_accum_steps=2,
)

100%|██████████| 314/314 [00:21<00:00, 14.90it/s]


Start A Accuracy: 0.1735668789808917


100%|██████████| 79/79 [00:05<00:00, 14.68it/s]


Start B Accuracy: 0.28662420382165604


51it [00:09,  5.64it/s]

...

Epoch 9, Step 299, Loss 0.0002871573669835925


314it [00:58,  5.38it/s]
100%|██████████| 314/314 [00:20<00:00, 15.09it/s]


Epoch 9 A Accuracy: 0.9952229299363057


100%|██████████| 79/79 [00:05<00:00, 14.55it/s]


Epoch 9 B Accuracy: 0.39490445859872614
